In [6]:
import sklearn.metrics
import numpy as np
import pandas as pd

In [63]:
def true_positives(y_true, y_pred, num_label=1):
    """
    """
    if type(y_true) == list:
        y_true = np.array(y_true)
    if type(y_pred) == list:
        y_pred = np.array(y_pred)
    
    return np.sum((y_true == num_label) & (y_pred == num_label))

def false_positives(y_true, y_pred, num_label=1):
    """
    """
    if type(y_true) == list:
        y_true = np.array(y_true)
    if type(y_pred) == list:
        y_pred = np.array(y_pred)
        
    return np.sum((y_true != num_label) & (y_pred == num_label))

def false_negatives(y_true, y_pred, num_label=1):
    """
    """
    if type(y_true) == list:
        y_true = np.array(y_true)
    if type(y_pred) == list:
        y_pred = np.array(y_pred)
        
    return np.sum((y_true == num_label) & (y_pred != num_label))

def true_negatives(y_true, y_pred, num_label=1):
    """
    """
    if type(y_true) == list:
        y_true = np.array(y_true)
    if type(y_pred) == list:
        y_pred = np.array(y_pred)
        
    return np.sum((y_true != num_label) & (y_pred != num_label))

def root_mean_squared_error(y_true, y_pred):
    """
    """
    return sklearn.metrics.mean_squared_error(y_true, y_pred, squared=False)


# From https://scikit-learn.org/stable/modules/model_evaluation.html
_metrics = {
    'classification': {
        'accuracy': sklearn.metrics.accuracy_score,
        'balanced_accuracy':sklearn.metrics.balanced_accuracy_score,
        'average_precision':sklearn.metrics.average_precision_score,
        'brier_score':sklearn.metrics.brier_score_loss,
        'f1':sklearn.metrics.f1_score,
        'log_loss':sklearn.metrics.log_loss,
        'precision':sklearn.metrics.precision_score,
        'recall':sklearn.metrics.recall_score,
        'jaccard':sklearn.metrics.jaccard_score,
        'matthews_corrcoef':sklearn.metrics.matthews_corrcoef,
        'roc_auc':sklearn.metrics.roc_auc_score,
        'true_positives':true_positives,
        'TP':true_positives,
        'false_positives':false_positives,
        'FP':false_positives,
        'false_negatives':false_negatives,
        'FN':false_negatives,
        'true_negatives':true_negatives,
        'TN':true_negatives,
        'confusion_matrix':sklearn.metrics.confusion_matrix
    },
    'regression':{
        'explained_variance': sklearn.metrics.explained_variance_score,
        'max_error': sklearn.metrics.max_error,
        'mean_absolute_error': sklearn.metrics.mean_absolute_error,
        'MAE': sklearn.metrics.mean_absolute_error,
        'mean_squared_error': sklearn.metrics.mean_squared_error,
        'MSE': sklearn.metrics.mean_squared_error,
        'root_mean_squared_error': root_mean_squared_error,
        'RMSE': root_mean_squared_error,
        'mean_squared_log_error': sklearn.metrics.mean_squared_log_error,
        'median_absolute_error': sklearn.metrics.median_absolute_error,
        'r2': sklearn.metrics.r2_score,
        'mean_poisson_deviance': sklearn.metrics.mean_poisson_deviance,
        'mean_gamma_deviance': sklearn.metrics.mean_gamma_deviance
    }
} 


In [55]:
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]

i=2
print(sklearn.metrics.confusion_matrix(y_true,y_pred))
print(true_positives(y_true,y_pred,num_label=i))
print(false_positives(y_true,y_pred,num_label=i))
print(false_negatives(y_true,y_pred,num_label=i))
print(true_negatives(y_true,y_pred,num_label=i))


[[2 0 0]
 [0 0 1]
 [1 0 2]]
2
1
1
2


In [57]:
y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0, 1, 0, 1, 0, 1, 0, 1]

for name, metric in metrics['classification'].items():
    print(name, metric(y_true, y_pred))

accuracy 0.625
balanced_accuracy 0.6333333333333333
average_precision 0.7
brier_score 0.375
f1 0.6666666666666665
log_loss 12.95214109777028
precision 0.75
recall 0.6
jaccard 0.5
matthews_corrcoef 0.2581988897471611
roc_auc 0.6333333333333334
true_positives 3
TP 3
false_positives 1
FP 1
false_negatives 2
FN 2
true_negatives 2
TN 2
confusion_matrix [[2 1]
 [2 3]]


In [50]:
y_true = [3, 5, 2.5, 7]
y_pred = [2.5, 5, 4, 8]
for name, metric in metrics['regression'].items():
    print(name, metric(y_true, y_pred))

explained_variance 0.8029556650246306
max_error 1.5
mean_absolute_error 0.75
mean_squared_error 0.875
root_mean_squared_error 0.9354143466934853
mean_squared_log_error 0.03973012298459379
median_absolute_error 0.75
r2 0.7241379310344828
mean_poisson_deviance 0.21861792444793338
mean_gamma_deviance 0.06060673253815174


In [98]:
import warnings

def compute_metrics(y_true, y_pred, metrics, task='classification'):
    """
    """
    if type(metrics) != list:
        raise TypeError('metrics must be a list')
        
    if type(y_true) == list:
        y_true = np.array(y_true)
    if type(y_pred) == list:
        y_pred = np.array(y_pred)
        
    fn_dict = {}
    cnt_custom = 1

    for fn in metrics:
        if type(fn) == str:
            if fn in _metrics[task]:
                fn_dict[fn] = _metrics[task][fn]
            else:
                warnings.warn('%s function not found' % fn, Warning)
        else:
            fn_dict['custom_'+str(cnt_custom)] = fn
            cnt_custom += 1
    metrics = fn_dict
    
    if len(metrics.keys()) == 0:
        raise ValueError('No valid metrics found')
                    
    if task == 'classification':
        y_score = y_pred
        y_pred = np.round(y_pred, 0)
    
    res = {}
    for name, fn in metrics.items():
        if (task == 'classification') & (
            name in ['average_precision','roc_auc']):
            
            res[name] = fn(y_true, y_score)
            continue
        
        res[name] = fn(y_true, y_pred)
        
    return res

y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0, 1, 0, 1, 0, 1, 0, 1]
metrics = ['accuracy','f1','lol',lambda y_true, y_pred:1]

print(compute_metrics(y_true, y_pred, metrics))


y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0.2, 0.49, 0, 1, 0.3, 0.8, 0, 1]

print(compute_metrics(y_true, y_pred, metrics))

y_true = [3, 5, 2.5, 7]
y_pred = [2.5, 5, 4, 8]
metrics = ['RMSE','MAE','lol',lambda y_true, y_pred:1]

print(compute_metrics(y_true, y_pred, metrics, task='regression'))

{'accuracy': 0.625, 'f1': 0.6666666666666665, 'custom_1': 1}
{'accuracy': 0.75, 'f1': 0.7499999999999999, 'custom_1': 1}
{'RMSE': 0.9354143466934853, 'MAE': 0.75, 'custom_1': 1}


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:22: Warning: lol function not found


In [89]:
for n, fn in _metrics['classification'].items():
    h = str(fn.__code__.co_varnames)
    if 'y_score' in h:
        print('score', n)
    if 'y_pred' in h:
        print('pred', n)

pred accuracy
pred balanced_accuracy
score average_precision
pred f1
pred log_loss
pred precision
pred recall
pred jaccard
pred matthews_corrcoef
score roc_auc
pred true_positives
pred TP
pred false_positives
pred FP
pred false_negatives
pred FN
pred true_negatives
pred TN
pred confusion_matrix


In [ ]:
y_true = [0, 0, 0, 1, 1, 1, 1, 1]
y_pred = [0.2, 0.9, 0, 1, 0.3, 0.8, 0, 1]